# Cohomology Ring Computation for Real Parabolic Arrangements

This notebook demonstrates the computation of cohomology rings for complements of real parabolic arrangements.

**Reference**: J. L. León-Medina and J. Cantarero, "The Topology of Real Parabolic Arrangements", manuscript in preparation.

---

## Overview

The implementation computes:
- Betti numbers of arrangement complements via restricted Coxeter permutahedra
- Cohomology bases with explicit cocycle representatives
- Cup product structure via the zonotopal (Serre) diagonal
- DGA structure validation (Leibniz rule)

## 1. Load Module

In [ ]:
load('parabolic_arrangement_cohomology.sage')

## 2. Minimal Example: A₃ (3-Equal Arrangement)

We start with a small example to understand the basic workflow.

The **3-equal arrangement** in type A₃ excludes parabolic cosets `wW_I` where `I` contains 3 consecutive simple reflections.

In [3]:
# Build Weyl group and parabolic coset poset
W, Plist, _ = build_W_P('A', 3)

print(f"Weyl group: {W.cartan_type()}")
print(f"Order of W: {W.cardinality()}")
print(f"Total parabolic cosets: {len(Plist)}")

Weyl group: ['A', 3]
Order of W: 24
Total parabolic cosets: 75


In [4]:
# Define the 3-equal arrangement
Delta = ideal_k_parabolic(W, Plist, k=3)

print(f"Excluded cells: {len(Delta)}")
print(f"Cells in complement: {len(Plist) - len(Delta)}")

Excluded cells: 9
Cells in complement: 66


In [5]:
# Initialize the cohomology ring computation
arrangement = ParabolicArrangementCohomology(W, Plist, Delta)

print("\nCellular complex dimensions:")
for k in range(arrangement.max_grade + 1):
    n_k = len(arrangement.by_grade.get(k, []))
    print(f"  C^{k}: {n_k} cells")


Cellular complex dimensions:
  C^0: 24 cells
  C^1: 36 cells
  C^2: 6 cells


## 3. Compute Betti Numbers

The cohomology `H^k` is computed as `ker(δ^k) / im(δ^{k-1})`.

In [6]:
print("Computing cohomology over F₂...\n")

betti_numbers = []
for k in range(arrangement.max_grade + 1):
    basis = arrangement.cohomology_basis(k, ring=GF(2))
    betti_numbers.append(len(basis))
    print(f"dim H^{k} = {len(basis)}")

print(f"\nBetti sequence: {tuple(betti_numbers)}")
print("Note: H² = 0 for A₃ (3-equal)")

Computing cohomology over F₂...

dim H^0 = 1
dim H^1 = 7
dim H^2 = 0

Betti sequence: (1, 7, 0)
Note: H² = 0 for A₃ (3-equal)


## 4. Verify DGA Structure

We verify that the coboundary operator and cup product satisfy the graded Leibniz rule:

$$\delta(u \cup v) = (\delta u) \cup v + (-1)^{|u|} u \cup (\delta v)$$

In [7]:
print("Verifying DGA structure...\n")
result = arrangement.verify_leibniz_rule(ring=GF(2), trials=10, use_fast=True)

if result:
    print("\nDGA structure verified!")
else:
    print("\nVerification failed!")

Verifying DGA structure...

--- Verifying DGA Leibniz Rule over Finite Field of size 2 ---
Testing degrees (1, 1)...
Leibniz Rule Verified: The DGA structure is consistent.

DGA structure verified!


## 5. Advanced Example: A₅ (3-Equal Arrangement)

For A₅, we have non-trivial H² and can compute cup products. This also allows us to verify Baryshnikov's theorem: H² is generated by cup products of H¹.

**Note**: This computation takes several seconds due to the size of the arrangement.

In [8]:
print("Building A₅ (k=3) arrangement...")
W5, Plist5, _ = build_W_P('A', 5)
Delta5 = ideal_k_parabolic(W5, Plist5, k=3)
arrangement5 = ParabolicArrangementCohomology(W5, Plist5, Delta5)

print(f"\nOrder of W: {W5.cardinality()}")
print(f"Total parabolic cosets: {len(Plist5)}")
print(f"Cells in complement: {len(Plist5) - len(Delta5)}")

Building A₅ (k=3) arrangement...

Order of W: 720
Total parabolic cosets: 4683
Cells in complement: 3690


In [9]:
print("Computing cohomology over F₂...\n")

for k in range(min(3, arrangement5.max_grade + 1)):
    basis = arrangement5.cohomology_basis(k, ring=GF(2))
    print(f"dim H^{k} = {len(basis)}")

print("\nExpected Betti numbers: (1, 111, 20)")

Computing cohomology over F₂...

dim H^0 = 1
dim H^1 = 111
dim H^2 = 20

Expected Betti numbers: (1, 111, 20)


## 6. Cup Product Computation

The cup product is computed using the zonotopal diagonal:

$$(u \cup v)(wW_I) = \sum_{J \sqcup K = I} \varepsilon(J,K) \cdot u(wW_J) \cdot v(w \cdot w_{0,J} W_K)$$

where $w_{0,J}$ is the longest element in the parabolic subgroup $W_J$.

In [10]:
# Get basis of H¹ for A₅
H1_basis = arrangement5.cohomology_basis(1, ring=GF(2))

print(f"H¹ has {len(H1_basis)} generators")

if len(H1_basis) >= 2:
    # Take first two generators
    u, v = H1_basis[0], H1_basis[1]
    
    print(f"\nComputing u₀ ∪ u₁...")
    print(f"  Support size of u₀: {u.hamming_weight()}")
    print(f"  Support size of u₁: {v.hamming_weight()}")
    
    # Compute cup product
    product = arrangement5.fast_cup_product(u, v, 1, 1, ring=GF(2))
    
    print(f"\nProduct u₀ ∪ u₁:")
    print(f"  Support size: {product.hamming_weight()}")
    print(f"  Is zero: {product.is_zero()}")

H¹ has 111 generators

Computing u₀ ∪ u₁...
  Support size of u₀: 23
  Support size of u₁: 20

Product u₀ ∪ u₁:
  Support size: 4
  Is zero: False


### Validation: H² Generation

We verify that H² is generated by cup products from H¹ (Baryshnikov's description for k-equal arrangements).

In [11]:
print("Analyzing cup product generation...\n")
result = arrangement5.cup_product_span_analysis(ring=GF(2), verbose=True)

Analyzing cup product generation...

CUP PRODUCT SPAN ANALYSIS

Computing cohomology bases...
  dim(H^1) = 111
  dim(H^2) = 20

Precomputing cup structure for (1,1)...
Building projection matrix...

Analyzing 6105 pairs...
[Hit #1] Pair (10, 2) -> Rank 1/20
[Hit #2] Pair (24, 20) -> Rank 2/20
[Hit #3] Pair (27, 21) -> Rank 3/20
[Hit #4] Pair (30, 31) -> Rank 4/20
[Hit #5] Pair (90, 87) -> Rank 5/20
[Hit #6] Pair (92, 88) -> Rank 6/20
[Hit #7] Pair (95, 96) -> Rank 7/20
[Hit #8] Pair (105, 106) -> Rank 8/20
[Hit #9] Pair (3, 7) -> Rank 9/20
[Hit #10] Pair (8, 36) -> Rank 10/20
[Hit #11] Pair (49, 22) -> Rank 11/20
[Hit #12] Pair (6, 35) -> Rank 12/20
[Hit #13] Pair (32, 18) -> Rank 13/20
[Hit #14] Pair (33, 14) -> Rank 14/20
[Hit #15] Pair (89, 50) -> Rank 15/20
[Hit #16] Pair (97, 57) -> Rank 16/20
[Hit #17] Pair (98, 53) -> Rank 17/20
[Hit #18] Pair (107, 83) -> Rank 18/20
[Hit #19] Pair (108, 86) -> Rank 19/20
[Hit #20] Pair (109, 78) -> Rank 20/20

SUCCESS: H^2 fully generated in 23

In [12]:
print("\n" + "=" * 60)
print("SUMMARY")
print("=" * 60)
print(f"H² dimension: {result['dim_H2']}")
print(f"Rank generated by products: {result['rank_generated']}")
print(f"Is complete: {result['is_complete']}")
print(f"Computation time: {result['time']:.2f}s")
print(f"Number of generating pairs: {len(result['generating_pairs'])}")

if result['is_complete']:
    print("\nBaryshnikov's theorem verified: H² is generated by H¹ products")
else:
    print(f"\nOnly {result['rank_generated']}/{result['dim_H2']} dimensions generated")


SUMMARY
H² dimension: 20
Rank generated by products: 20
Is complete: True
Computation time: 23.75s
Number of generating pairs: 20

Baryshnikov's theorem verified: H² is generated by H¹ products


## 7. Other Weyl Group Types

The theory and implementation work for all finite Weyl groups. Here are some examples:

In [13]:
# Type B₃ example
print("Type B₃ (hyperoctahedral group):")
WB, PlistB, _ = build_W_P('B', 3)
DeltaB = ideal_k_parabolic(WB, PlistB, k=3)
arrangementB = ParabolicArrangementCohomology(WB, PlistB, DeltaB)

for k in range(min(3, arrangementB.max_grade + 1)):
    basis = arrangementB.cohomology_basis(k, ring=GF(2))
    print(f"  dim H^{k} = {len(basis)}")

Type B₃ (hyperoctahedral group):
  dim H^0 = 1
  dim H^1 = 13
  dim H^2 = 0


In [14]:
# Type D₄ example
print("\nType D₄ (even signed permutations):")
WD, PlistD, _ = build_W_P('D', 4)
DeltaD = ideal_k_parabolic(WD, PlistD, k=3)
arrangementD = ParabolicArrangementCohomology(WD, PlistD, DeltaD)

for k in range(min(3, arrangementD.max_grade + 1)):
    basis = arrangementD.cohomology_basis(k, ring=GF(2))
    print(f"  dim H^{k} = {len(basis)}")


Type D₄ (even signed permutations):
  dim H^0 = 1
  dim H^1 = 73
  dim H^2 = 0


## Summary

This notebook demonstrates:

1. Construction of k-parabolic arrangements from Weyl groups
2. Computation of Betti numbers via cellular cohomology
3. Verification of DGA structure (graded Leibniz rule)
4. Cup product computation via zonotopal diagonal
5. Validation of Baryshnikov's generation theorem
6. Examples in types A, B, D

### Key Features

- **Efficient caching**: Geometric structures are precomputed and reused
- **General theory**: Works for arbitrary parabolic arrangements in any finite Weyl group
- **Explicit formulas**: Direct cellular computation without barycentric subdivision

---

### Citation

If you use this code in your research, please cite:

> J. L. León-Medina and J. Cantarero, "The Topology of Real Parabolic Arrangements", manuscript in preparation.

---

**Version**: 1.2 | **Updated**: February 2025